### Notebook to go through all mat files and get hand labeled trials that have 3 or more behaviors labeled

In [1]:
from pathlib import Path
from tqdm import tqdm
import shutil
import os
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import pandas as pd

In [2]:
columns = ["mat_path", "hand_labels"]
df = pd.DataFrame(columns=columns)

In [3]:
df

,mat_path,hand_labels


In [4]:
mat_dir = Path("/data/caitlin/mat_files/")

In [5]:
def check_all_behaviors(ethogram: np.ndarray):
    count = 0
    for row in ethogram:
        if row.any():
            count += 1
    if count >= 3:
        return True
    else:
        return False

In [6]:
mat_paths = sorted(mat_dir.glob('*'))
len(mat_paths)

696

In [7]:
mat_errors = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data"]["trial"].shape[0]
        
    except KeyError:
        mat_errors.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            continue

        behaviors = sorted([b.split('_')[0] for b in m['data'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data'].dtype.names.index(f'{b}_labl_label')
            row = m['data'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [8]:
for mat in tqdm(mat_paths):
    hand_labels = get_hand_labels(str(mat))
    if hand_labels:
        df.loc[len(df.index)] = [mat.stem, hand_labels] 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5562.00it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 5356.39it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5639.50it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5616.27it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5513.92it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5623.99it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5497.93it/s]


In [9]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v022': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v006': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170310,"{'M232_20170310_v013': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M232_20170314,"{'M232_20170314_v011': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
287,M336_20210612,"{'M336_20210612_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
288,M336_20210613,"{'M336_20210613_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
289,M336_20210614,"{'M336_20210614_v019': [[0, 0, 0, 0, 0, 0, 0, ..."
290,M336_20210615,"{'M336_20210615_v001': [[0, 0, 0, 0, 0, 0, 0, ..."


In [10]:
mat_errors

['/data/caitlin/mat_files/M236_20170817.mat',
 '/data/caitlin/mat_files/M236_20170818.mat',
 '/data/caitlin/mat_files/M238_20170717.mat',
 '/data/caitlin/mat_files/M238_20170719.mat',
 '/data/caitlin/mat_files/M238_20170724.mat',
 '/data/caitlin/mat_files/M238_20170725.mat',
 '/data/caitlin/mat_files/M238_20170726.mat',
 '/data/caitlin/mat_files/M238_20170727.mat',
 '/data/caitlin/mat_files/M240_20170724FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170725FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170727FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170728FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170801FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170802FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170807FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170808FinalChecked.mat']

In [11]:
count = 0
for row in df.iterrows():
    count += len(row[1]["hand_labels"].keys())
count

1253

In [12]:
df.to_hdf(path_or_buf="/data/caitlin/3ormore_hand_labels.hdf", key="df")

/tmp/ipykernel_3355608/1933448500.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/3ormore_hand_labels.hdf", key="df")


In [13]:
df = pd.read_hdf("/data/caitlin/3ormore_hand_labels.hdf")

In [14]:
mat_errors2 = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data1"]["trial"].shape[0]
        
    except KeyError:
        mat_errors2.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data1"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            continue

        behaviors = sorted([b.split('_')[0] for b in m['data1'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data1'].dtype.names.index(f'{b}_labl_label')
            row = m['data1'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data1"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [15]:
for mat in tqdm(mat_errors):
    hand_labels = get_hand_labels(Path(mat))
    if hand_labels:
        df.loc[len(df.index)] = [Path(mat).stem, hand_labels] 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5624.18it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5672.00it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5655.94it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5709.84it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5685.17it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5677.86it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5720.55it/s]


In [16]:
mat_errors2

[]

In [17]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v022': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v006': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170310,"{'M232_20170310_v013': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M232_20170314,"{'M232_20170314_v011': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
294,M238_20170717,"{'M238_20170717_v021': [[0, 0, 0, 0, 0, 0, 0, ..."
295,M238_20170725,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
296,M238_20170726,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
297,M240_20170725FinalChecked,"{'M240_20170725_v084': [[0, 0, 0, 0, 0, 0, 0, ..."


In [18]:
df.to_hdf(path_or_buf="/data/caitlin/3ormore_hand_labels.hdf", key="df")

/tmp/ipykernel_3355608/1933448500.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/3ormore_hand_labels.hdf", key="df")


In [19]:
df = pd.read_hdf("/data/caitlin/3ormore_hand_labels.hdf")

In [20]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v022': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v006': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170310,"{'M232_20170310_v013': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M232_20170314,"{'M232_20170314_v011': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
294,M238_20170717,"{'M238_20170717_v021': [[0, 0, 0, 0, 0, 0, 0, ..."
295,M238_20170725,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
296,M238_20170726,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
297,M240_20170725FinalChecked,"{'M240_20170725_v084': [[0, 0, 0, 0, 0, 0, 0, ..."


In [21]:
count = 0
for row in df.iterrows():
    count += len(row[1]["hand_labels"].keys())
count

1265